In [1]:
# test code to scrape additional information

In [2]:
import time
from csv import DictWriter
from datetime import date

from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import hkjobsdb_date_reformat

TARGET_JOB = 'data-scientist'
WEBSITE = f'https://hk.jobsdb.com/hk/job-list/information-technology/{TARGET_JOB}/'
LOAD_SEC_PER_JOB = 3      # increase if internet is slow


In [3]:
options = Options()
options.add_argument("start-maximized")
# options.add_argument('headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(WEBSITE+'1')

In [4]:
time.sleep(LOAD_SEC_PER_JOB)
left_panel_job_link_list = driver.find_elements(By.XPATH, '//div[@class="sx2jih0 zcydq8n lmSnC_0"]')
job_link = left_panel_job_link_list[1]
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(job_link)).click()
except exceptions.ElementClickInterceptedException:
    print('job not clickable')
    pass                

In [5]:
time.sleep(LOAD_SEC_PER_JOB)
try:
    title = driver.find_element(By.XPATH, '//h1[@class ="sx2jih0 _18qlyvc0 _18qlyvch _1d0g9qk4 _18qlyvcp _18qlyvc1x"]').text
except exceptions.NoSuchElementException:
    title = 'title'     # scrape fail/missing value placeholder   
title

'Data Analyst, 35k, Perm, Local Listed Media Company'

In [6]:
time.sleep(5)
# additional information table
# <div class="sx2jih0 zcydq856 zcydq8fe _17fduda27">

try:
    additional_table = driver.find_element(By.XPATH, '//div[@class ="sx2jih0 zcydq856 zcydq8fe _17fduda27"]')
except exceptions.NoSuchElementException:
    pass   # scrape fail/missing value placeholder   
additional_table

<selenium.webdriver.remote.webelement.WebElement (session="837125fee27153873c488e55cefaea04", element="3b9f8238-9a74-4875-88db-891fcf7758aa")>

In [7]:
def scrap_additional_info_as_dict():
    time.sleep(5)
    # additional information table
    # <div class="sx2jih0 zcydq856 zcydq8fe _17fduda27">

    try:
        additional_table = driver.find_element(By.XPATH, '//div[@class ="sx2jih0 zcydq856 zcydq8fe _17fduda27"]')
    except exceptions.NoSuchElementException:
        pass   # scrape fail/missing value placeholder   

    additional_info_raw= additional_table.text
    additional_info_list = additional_info_raw.split('\n')
    table_len = len(additional_info_list)
    additional_columns = []
    additional_values = []
    for i in range(table_len):
        if i%2==0:
            additional_columns.append(additional_info_list[i])
        else:
            additional_values.append(additional_info_list[i])
    
    #print(addditional_columns)
    #print(additional_values)
    
    additional_info_dict = dict(map(lambda i,j : (i,j) , additional_columns,additional_values))
    return additional_info_dict
        

In [8]:
print(scrap_additional_info_as_dict())

{'Career Level': 'Entry Level', 'Qualification': 'Non-Degree Tertiary', 'Years of Experience': '2 years', 'Job Type': 'Full Time, Permanent', 'Job Functions': 'Media & Advertising, Editorial / Journalism, Information Technology (IT), Data Scientist'}


In [9]:
# test dictwrite 

In [10]:
from csv import DictWriter

columns_complete = ['0','1','2','3']
values_complete = ['a','b','c','d']
columns_incomplete = ['0','2','3']
values_incomplete = ['e','f','g']

def dict_gen(list_a, list_b):
    return dict(map(lambda i,j : (i,j) , list_a, list_b))

dict_1 = dict_gen(columns_complete, values_complete)
dict_2 = dict_gen(columns_incomplete, values_incomplete)

with open('dict_test.csv', 'w', newline='') as test_fh:
    writer = DictWriter(test_fh, fieldnames=columns_complete)
    
    writer.writeheader()
    writer.writerow(dict_1)
    writer.writerow(dict_2)